In [1]:
import pandas as pd
import numpy as np
import tabula
import re

import os
import glob
import time
from tkinter.filedialog import askdirectory

In [12]:
current_directory = os.getcwd()
pdf_files = glob.glob(f'{current_directory}/*.pdf')
area = [0, 0, 1000, 1000]

for pdf_file in pdf_files:
    print(pdf_file)
    dfs = tabula.read_pdf(pdf_file, area=area, pages='all')

    df = pd.concat(dfs)
    df.fillna('', inplace=True)

    invoice_mask = df['Unnamed: 0'].str.contains('Invoice No. / Date')
    date_index = df[invoice_mask].index.values[0]
    invoice_date = df.loc[date_index, 'Invoice']
    invoice_reg = r'\d+[^ ]*'

    invoice_num = re.findall(invoice_reg, invoice_date)[0]
    date = re.findall(invoice_reg, invoice_date)[1]

    acc_mask = df['Unnamed: 0'].str.contains(' Purchase Order No.')
    acc_index = df[acc_mask].index.values[0] 
    acc_num = df.loc[acc_index, 'Invoice']
    acc_reg = r'(?<=\s)\d+'
    acc_num = re.findall(acc_reg, acc_num)[0]

    total_mask = df["Invoice"].str.contains('Qty Total Curr.')
    start = df[total_mask].index.values[0]
    total = df.loc[start + 1, 'Invoice']
    total_reg = r"(?<=\$)\d+\.\d+"
    total = re.findall(total_reg, total)[0]

    data = {
        'Date': date,
        'Invoice': invoice_num,
        'Account': acc_num,
        'Total': total
    }

    df1 = pd.DataFrame(data, index=['0'])

    df1.to_excel(f'{df1.Account[0]}.xlsx', index=False)
    print(pdf_file)


         Date     Invoice  Account  Total
0  05/11/2023  6915077305  8003603  43.66
/Users/joshburch/codeup-data-science/Web_Scraping/6915077305.pdf
         Date     Invoice  Account   Total
0  05/11/2023  6915075704  8003606  152.30
/Users/joshburch/codeup-data-science/Web_Scraping/6915075704.pdf
         Date     Invoice  Account   Total
0  05/10/2023  6915070718  1246910  138.96
/Users/joshburch/codeup-data-science/Web_Scraping/6915070718.pdf


In [12]:
current_directory = os.getcwd()
pdf_files = glob.glob(f'{current_directory}/*.pdf')
area = [0, 0, 1000, 1000]

for pdf_file in pdf_files:  
    dfs = tabula.read_pdf(pdf_file, area=area, pages='all')

    df = pd.concat(dfs)
    df.fillna('', inplace=True)

    invoice_mask = df['Unnamed: 0'].str.contains('Invoice No. / Date')
    date_index = df[invoice_mask].index.values[0]
    invoice_date = df.loc[date_index, 'Invoice']
    invoice_reg = r'\d+[^ ]*'

    invoice_num = re.findall(invoice_reg, invoice_date)[0]
    date = re.findall(invoice_reg, invoice_date)[1]

    acc_mask = df['Unnamed: 0'].str.contains(' Purchase Order No.')
    acc_index = df[acc_mask].index.values[0] 
    acc_num = df.loc[acc_index, 'Invoice']
    acc_reg = r'(?<=\s)\d+'
    acc_num = re.findall(acc_reg, acc_num)[0]

    total_mask = df["Invoice"].str.contains('Qty Total Curr.')
    start = df[total_mask].index.values[0]
    total = df.loc[start + 1, 'Invoice']
    total_reg = r"(?<=\$)\d+\.\d+"
    total = re.findall(total_reg, total)[0]

    data = {
        'Date': date,
        'Invoice': invoice_num,
        'Account': acc_num,
        'Total': total
    }

    df1 = pd.DataFrame(data, index=['0'])

    df1.to_excel(f'{df1.Account[0]}.xlsx', index=False)
    print(pdf_file)


         Date     Invoice  Account  Total
0  05/11/2023  6915077305  8003603  43.66
/Users/joshburch/codeup-data-science/Web_Scraping/6915077305.pdf
         Date     Invoice  Account   Total
0  05/11/2023  6915075704  8003606  152.30
/Users/joshburch/codeup-data-science/Web_Scraping/6915075704.pdf
         Date     Invoice  Account   Total
0  05/10/2023  6915070718  1246910  138.96
/Users/joshburch/codeup-data-science/Web_Scraping/6915070718.pdf


In [11]:
df1

,Date,Invoice,Account,Total
0,05/10/2023,6915070718,1246910,138.96


In [123]:
def read_pdfs():
    current_directory = '/Users/joshburch/projects/pdf-reader/pdf/93pdf'
    pdf_files = glob.glob(f'{current_directory}/*.pdf')
    area = [0, 0, 1000, 1000]
    invoice_reg = r'\d+[^ ]*'
    acc_reg = r'(?<=\s)\d+'
    total_reg= r'\$(\d+(?:,\d+)?(?:\.\d+)?)'

    for pdf_file in pdf_files:
        print(pdf_file)
        dfs = tabula.read_pdf(pdf_file, area=area, pages='all')

        df = pd.concat(dfs)
        df.fillna('', inplace=True)
        df = df.drop_duplicates()
        is_invoice = df.isin(['Invoice No. / Date']).any().any()

        if is_invoice:
            doc_type = 'Invoice'
            invoice_column = df.columns[df.isin(['Invoice No. / Date']).any()][0]
            invoice_mask = df[invoice_column].str.contains('Invoice No. / Date')
            date_index = df[invoice_mask].index.values[0]
            invoice_date = df.loc[date_index, 'Invoice']
            acc_column = df.columns[df.isin(['Invoice No. / Date']).any()][0]
            acc_mask = df[acc_column].str.contains('Purchase Order No.')
            acc_index = df[acc_mask].index.values[0]
            try:
                acc_num = df.loc[acc_index, 'Invoice']
                acc_num = re.findall(acc_reg, acc_num)[0]
            except TypeError:
                acc_num = df.loc[acc_index, 'Invoice'].values[0]
                acc_num = re.findall(acc_reg, acc_num)[0]
            total_mask = df["Invoice"].str.contains('Qty Total Curr.')
            start = df[total_mask].index.values[0]
            total = df.loc[start + 1, 'Invoice']
            start = df[total_mask].index.values[0]
            total = re.findall(total_reg, total)[0]  

        else:
            doc_type = 'Returns Credit Memo'
            invoice_mask = df['Unnamed: 1'].str.contains('Credit N. / Date')
            date_index = df[invoice_mask].index.values[0]
            invoice_date = df.loc[date_index, 'Returns Credit Memo']
            acc_mask = df['Unnamed: 0'].str.contains('Bill-To Address')
            acc_index = df[acc_mask].index.values[0] + 1 
            acc_num = df.loc[acc_index, 'Unnamed: 0']
            total_mask = df["Unnamed: 0"].str.contains('Credit Note Summary')
            start = df[total_mask].index.values[0]
            total = df.loc[start + 1, 'Returns Credit Memo']
            total = '-' + re.findall(total_reg, total)[0]


        try:
            invoice_num = re.findall(invoice_reg, invoice_date)[0]
        except TypeError:
            invoice_date = invoice_date.values[0]
            invoice_num = re.findall(invoice_reg, invoice_date)[0]

        date = re.findall(invoice_reg, invoice_date)[1]



        data = {
            'Date': date,
            'Doc_type': doc_type,
            'Invoice': invoice_num,
            'Account': acc_num,
            'Total': total
        }
        df1 = pd.DataFrame(data, index=['0'])
        
        try:
            df2 = pd.read_excel(f'{acc_num}.xlsx')
            df3 = pd.concat([df2, df1], ignore_index=True)
            df3.to_excel(f'{df1.Account[0]}.xlsx', index=False)
        except FileNotFoundError:
            df1.to_excel(f'{df1.Account[0]}.xlsx', index=False)

In [124]:
read_pdfs()

/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915801235_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915719615_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915605797_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915706115_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915760507_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915615395_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915726096_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915676958_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915725670_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915584514_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915679251_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915771654_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915771655_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915661294_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/6915769094_ZFU3

In [126]:
pd.read_excel('8003602.xlsx')

,Date,Doc_type,Invoice,Account,Total
0,07/20/2023,Invoice,6915726096,8003602,60.21
1,07/26/2023,Invoice,6915769094,8003602,76.12
2,07/14/2023,Invoice,6915675504,8003602,37.06
3,08/01/2023,Invoice,6915824945,8003602,88.42
4,08/02/2023,Invoice,6915835178,8003602,52.56
5,07/20/2023,Invoice,6915726101,8003602,69.61
6,07/14/2023,Invoice,6915670969,8003602,71.17
7,07/02/2023,Invoice,6915578423,8003602,52.56
8,07/11/2023,Invoice,6915645251,8003602,71.17
9,07/22/2023,Invoice,6915740904,8003602,88.42


In [33]:
ask = askdirectory()

2023-07-02 08:47:05.281 python[43041:15407361] +[CATransaction synchronize] called within transaction
2023-07-02 08:47:05.286 python[43041:15407361] +[CATransaction synchronize] called within transaction
2023-07-02 08:47:05.302 python[43041:15407361] +[CATransaction synchronize] called within transaction
2023-07-02 08:47:05.342 python[43041:15407361] +[CATransaction synchronize] called within transaction
2023-07-02 08:47:05.350 python[43041:15407361] +[CATransaction synchronize] called within transaction
2023-07-02 08:47:08.409 python[43041:15407361] +[CATransaction synchronize] called within transaction


In [34]:
ask

'/Users/joshburch/Desktop/output'

In [22]:
pdf_files

['/Users/joshburch/codeup-data-science/Web_Scraping/6915077305.pdf',
 '/Users/joshburch/codeup-data-science/Web_Scraping/6915075704.pdf',
 '/Users/joshburch/codeup-data-science/Web_Scraping/6915070718.pdf']

In [2]:
current_directory = os.getcwd()

In [3]:
current_directory

'/Users/joshburch/projects/pdf-reader'

In [4]:
current_directory = '/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/'

In [38]:
acc = re.findall(acc_reg, acc_num)

In [50]:
is_invoice

True

In [71]:
date_index

1

In [121]:
current_directory = '/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/'
pdf_files = glob.glob(f'{current_directory}/*.pdf')
area = [0, 0, 1000, 1000]
invoice_reg = r'\d+[^ ]*'
acc_reg = r'(?<=\s)\d+'
total_reg= r'\$(\d+(?:,\d+)?(?:\.\d+)?)'

for pdf_file in pdf_files:
    print(pdf_file)
    dfs = tabula.read_pdf(pdf_file, area=area, pages='all')

    df = pd.concat(dfs)
    df.fillna('', inplace=True)
    df = df.drop_duplicates()
    is_invoice = df.isin(['Invoice No. / Date']).any().any()
    
    if is_invoice:
        doc_type = 'Invoice'
        invoice_column = df.columns[df.isin(['Invoice No. / Date']).any()][0]
        invoice_mask = df[invoice_column].str.contains('Invoice No. / Date')
        date_index = df[invoice_mask].index.values[0]
        invoice_date = df.loc[date_index, 'Invoice']
        acc_column = df.columns[df.isin(['Invoice No. / Date']).any()][0]
        acc_mask = df[acc_column].str.contains('Purchase Order No.')
        acc_index = df[acc_mask].index.values[0]
        try:
            acc_num = df.loc[acc_index, 'Invoice']
            acc_num = re.findall(acc_reg, acc_num)[0]
        except TypeError:
            acc_num = df.loc[acc_index, 'Invoice'].values[0]
            acc_num = re.findall(acc_reg, acc_num)[0]
        total_mask = df["Invoice"].str.contains('Qty Total Curr.')
        start = df[total_mask].index.values[0]
        total = df.loc[start + 1, 'Invoice']
        start = df[total_mask].index.values[0]
        total = re.findall(total_reg, total)[0]  
        
    else:
        doc_type = 'Returns Credit Memo'
        invoice_mask = df['Unnamed: 1'].str.contains('Credit N. / Date')
        date_index = df[invoice_mask].index.values[0]
        invoice_date = df.loc[date_index, 'Returns Credit Memo']
        acc_mask = df['Unnamed: 0'].str.contains('Bill-To Address')
        acc_index = df[acc_mask].index.values[0] + 1 
        acc_num = df.loc[acc_index, 'Unnamed: 0']
        total_mask = df["Unnamed: 0"].str.contains('Credit Note Summary')
        start = df[total_mask].index.values[0]
        total = df.loc[start + 1, 'Returns Credit Memo']
        total = '-' + re.findall(total_reg, total)[0]
                         
        
    try:
        invoice_num = re.findall(invoice_reg, invoice_date)[0]
    except TypeError:
        invoice_date = invoice_date.values[0]
        invoice_num = re.findall(invoice_reg, invoice_date)[0]
        
    date = re.findall(invoice_reg, invoice_date)[1]

   

    data = {
        'Date': date,
        'Doc_type': doc_type,
        'Invoice': invoice_num,
        'Account': acc_num,
        'Total': total
    }

    df1 = pd.DataFrame(data, index=['0'])

    df1.to_excel(f'{df1.Account[0]}.xlsx', index=False)



/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915605797_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915615395_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915676958_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915584514_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915661294_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915675502_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915660837_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915591377_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915675504_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915597090_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915595044_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915653130_ZFU3.pdf
/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915599588_ZFU3.pdf
/Users/joshburch/projects

In [119]:
invoice_date.values[0]

'6915657248 / 07/12/2023 Ship to Address'

In [8]:
dfs = tabula.read_pdf('/Users/joshburch/projects/pdf-reader/pdf/93pdf/batch01/6915676958_ZFU3.pdf', area=area, pages='all')

df = pd.concat(dfs)

In [83]:
df.head(50)

,Unnamed: 0,Unnamed: 1,Invoice,Unnamed: 2
0,LUXOTTICA OF AMERICA INC.,,,
1,Invoice No. / Date,,6915657248 / 07/12/2023 Ship to Address,
2,12 Harbor Park Drive Purchase Order No.,,1689081122560 1246910,
3,"Port Washington, NY 11050 Order",,183945311 / 07/11/2023,
4,,,FAMILY VISION CARE,
5,U.S.A. Payer,,1246910 4801 WEST BETHEL AVE,
6,Tel: 800-422-2020 Order Type,,ZA00 MUNCIE IN 47304,
7,Delivery No. / Date,,8831965529 / 07/12/2023,
8,MY.LUXOTTICA.COM,,,
9,D-U-N-S-79-101-3642,,,


In [34]:
total

'-538.78'